**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df = pd.read_csv("../../../Datasets/Flows/iomt_flows.csv")
display(df)

/project/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,traffic,label
0,10.10.10.252,33540,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,73,0,0,-,0.0,1,0,1,0,0.0,0.0,0.0,0.0,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
1,10.10.10.252,50435,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.0,1,0,1,0,0.0,0.0,0.0,0.0,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0,46.0,46.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,46.0,46.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
2,10.10.10.252,47976,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,57,0,0,-,0.0,1,0,1,0,0.0,0.0,0.0,0.0,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
3,10.10.10.252,37995,10.10.10.0,137,udp,dns,-,-,-,S0,T,T,0,D,1,78,0,0,-,0.0,1,0,1,0,0.0,0.0,0.0,0.0,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,50.0,50.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,50.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
4,10.10.10.252,38680,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,67,0,0,-,0.0,1,0,1,0,0.0,0.0,0.0,0.0,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,39.0,39.0,39.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,39.0,39.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4325373,42.32.107.34,32572,148.94.193.65,52545,tcp,-,-,-,-,OTH,F,F,0,-,0,0,0,0,-,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

Fix Dataframe Mixed Types

In [4]:
# Remove rows with '-' character in columns 7, 8 and 9
cols_to_check = ['duration', 'orig_bytes', 'resp_bytes']
#cols_to_check = ['duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'history', 'flow_duration']

mask = df[cols_to_check].apply(lambda x: x.str.contains('-', na=False)).any(axis=1)
df = df[~mask]

# Replace comma with period as decimal separator

cols_to_float = ['duration']
df[cols_to_float] = df[cols_to_float].replace(',', '.', regex=True)

# Convert columns 7, 8, 9, and 17 to float and int data type
cols_to_int = ['orig_bytes', 'resp_bytes']

df[cols_to_float] = df[cols_to_float].astype(float)
df[cols_to_int] = df[cols_to_int].astype(int)

-----------------------------------------------------------

**DF Statistics and Info**

In [5]:
def display_information_dataframe(df_cop):
    summary_data = [{'Data Type': dtype, 'Column Name': col, 'Unique Values': df_cop[col].unique()} for col, dtype in df_cop.dtypes.iteritems()]
    summary_df = pd.DataFrame(summary_data)
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    return summary_df

In [6]:
display_information_dataframe(df)

,Data Type,Column Name,Unique Values
0,object,id.orig_h,"[10.10.10.252, 10.10.10.1, 10.10.10.249, 0.0.0..."
1,int64,id.orig_p,"[5353, 36990, 45852, 55492, 59436, 60249, 5084..."
2,object,id.resp_h,"[224.0.0.251, 10.10.10.255, 10.10.10.249, 255...."
3,int64,id.resp_p,"[5353, 57621, 80, 22, 48969, 54845, 19582, 355..."
4,object,proto,"[udp, tcp, icmp]"
5,object,service,"[dns, -, http, mqtt, dhcp]"
6,float64,duration,"[57.262413, 3.002431, 0.208294, 0.209607, 0.20..."
7,int64,orig_bytes,"[1515, 75, 406, 405, 586, 569, 759, 763, 583, ..."
8,int64,resp_bytes,"[0, 851, 712, 310, 1111456671, 1287165066, 173..."
9,object,conn_state,"[S0, SH, RSTR, OTH, RSTRH, S2, RSTOS0, REJ, SH..."


--------------------------------------------

**Pre-processing and Data Encoding**

Split History

In [7]:
def count_letters(string, is_upper):
    count = 0
    for c in string:
        if is_upper and c.isupper():
            count += 1
        elif not is_upper and c.islower():
            count += 1
    return count

In [8]:
df['history_originator'] = df['history'].apply(lambda x: count_letters(x, True))
df['history_responder'] = df['history'].apply(lambda x: count_letters(x, False))

One Hot Encoding

In [9]:
def one_hot_encoding(df, columns):
    for col in columns:
        print(f'[ONE HOT ENCONDING] {col}')
        df = pd.get_dummies(df, columns=[col], prefix=col)
    return df

In [10]:
cols_to_encode = [
    'proto',
    'conn_state',
    'fwd_header_size_min',
    'fwd_header_size_max',
    'bwd_header_size_min',
    'bwd_header_size_max',
    'flow_FIN_flag_count',
    'flow_SYN_flag_count',
    'flow_RST_flag_count',
    'history_originator',
    'history_responder',
]

df = one_hot_encoding(df,cols_to_encode)

[ONE HOT ENCONDING] proto
[ONE HOT ENCONDING] conn_state
[ONE HOT ENCONDING] fwd_header_size_min
[ONE HOT ENCONDING] fwd_header_size_max
[ONE HOT ENCONDING] bwd_header_size_min
[ONE HOT ENCONDING] bwd_header_size_max
[ONE HOT ENCONDING] flow_FIN_flag_count
[ONE HOT ENCONDING] flow_SYN_flag_count
[ONE HOT ENCONDING] flow_RST_flag_count
[ONE HOT ENCONDING] history_originator
[ONE HOT ENCONDING] history_responder


In [11]:
def missed_bytes(missed_bytes):
    if missed_bytes < 1:
        return 0
    else:
        return 1

In [12]:
df['missed_bytes'] = df.apply(lambda row: missed_bytes(row['missed_bytes']), axis=1)

Remove Outliers

In [13]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVE OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [14]:
outliers = [
    'orig_pkts',
    'resp_pkts',
    'orig_ip_bytes',
    'resp_ip_bytes',
]

df = remove_outliers(df, outliers, 3)

[REMOVE OUTLIERS] orig_pkts
[REMOVE OUTLIERS] resp_pkts
[REMOVE OUTLIERS] orig_ip_bytes
[REMOVE OUTLIERS] resp_ip_bytes


Normalize, Z-score

In [17]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [18]:
cols_to_zscore = [
    'flow_duration', 'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
    'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
    'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot', 
    'bwd_header_size_tot', 'fwd_PSH_flag_count',
    'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_pkts_payload.min',
    'fwd_pkts_payload.max', 'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg',
    'fwd_pkts_payload.std', 'bwd_pkts_payload.min', 'bwd_pkts_payload.max',
    'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg', 'bwd_pkts_payload.std',
    'flow_pkts_payload.min', 'flow_pkts_payload.max', 'flow_pkts_payload.tot',
    'flow_pkts_payload.avg', 'flow_pkts_payload.std', 'fwd_iat.min',
    'fwd_iat.max', 'fwd_iat.tot', 'fwd_iat.avg', 'fwd_iat.std',
    'bwd_iat.min', 'bwd_iat.max', 'bwd_iat.tot', 'bwd_iat.avg', 'bwd_iat.std',
    'flow_iat.min', 'flow_iat.max', 'flow_iat.tot', 'flow_iat.avg',
    'flow_iat.std', 'payload_bytes_per_second', 'fwd_subflow_pkts',
    'bwd_subflow_pkts', 'fwd_subflow_bytes', 'bwd_subflow_bytes',
    'fwd_bulk_bytes', 'bwd_bulk_bytes', 'fwd_bulk_packets', 'bwd_bulk_packets',
    'fwd_bulk_rate', 'bwd_bulk_rate', 'active.max', 'active.tot',
    'active.avg', 'active.std', 'idle.min', 'idle.max', 'idle.tot',
    'idle.avg', 'idle.std', 'fwd_init_window_size', 'bwd_init_window_size',
    'fwd_last_window_size', 'bwd_last_window_size', 'duration', 'orig_bytes',
    'resp_bytes', 'orig_pkts', 'resp_pkts', 'resp_ip_bytes', 'orig_ip_bytes',
]

df = zscore_normalization(df, cols_to_zscore)

[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


---------------------------------------

**Create Model & Train Model**

In [19]:
df['is_attack'] = df['traffic'].apply(lambda x: 0 if x == "normal" else 1)
df.groupby('is_attack')['is_attack'].count()

is_attack
0         11
1    1843624
Name: is_attack, dtype: int64

Delete Insignificant Columns from the Dataframe

In [20]:
def delete_columns(df, cols):
    for col in cols:
        df.drop(col, axis = 1, inplace = True)
        print(f'[REMOVED] {col}')
    
    return df

In [21]:
cols_to_del = [
    'id.orig_h',
    'id.orig_p',
    'id.resp_h',
    'id.resp_p',
    'active.min',
    'service',
    'history',
    'traffic',
    'local_orig',
    'local_resp',
    'tunnel_parents',
    'fwd_URG_flag_count',
    'bwd_URG_flag_count',
    'flow_CWR_flag_count',
    'flow_ECE_flag_count',
    'traffic'
    ]

df = delete_columns(df,cols_to_del)

[REMOVED] id.orig_h
[REMOVED] id.orig_p
[REMOVED] id.resp_h
[REMOVED] id.resp_p
[REMOVED] active.min
[REMOVED] service
[REMOVED] history
[REMOVED] traffic
[REMOVED] local_orig
[REMOVED] local_resp
[REMOVED] tunnel_parents
[REMOVED] fwd_URG_flag_count
[REMOVED] bwd_URG_flag_count
[REMOVED] flow_CWR_flag_count
[REMOVED] flow_ECE_flag_count


KeyError: "['traffic'] not found in axis"

In [ ]:
x_columns = df.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df["is_attack"].values)

x = df[x_columns].values
y = df["is_attack"].values
y = le.transform(y)

In [ ]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [22]:
# Split into training and testing sets
x_train_train, x_test_train, y_train_train, y_test_train = train_test_split(x_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [23]:
x_test.shape,y_test.shape

((461334, 147), (461334,))

In [24]:
x_train.shape,y_train.shape

((1384001, 147), (1384001,))

In [25]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=x_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

# Define early stopping
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,mode="min",patience=10,verbose=1,min_lr=1e-7)
checkpoint = ModelCheckpoint('best_model_binary.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=512, callbacks=[monitor, checkpoint])

Train on 1384001 samples, validate on 461334 samples
Epoch 1/100
1384001/1384001 [==============================] - 7s 5us/sample - loss: 0.0296 - accuracy: 0.9996 - val_loss: 3.3597e-04 - val_accuracy: 1.0000
Epoch 2/100
1384001/1384001 [==============================] - 7s 5us/sample - loss: 4.1999e-04 - accuracy: 1.0000 - val_loss: 2.5653e-04 - val_accuracy: 1.0000
Epoch 3/100
1384001/1384001 [==============================] - 7s 5us/sample - loss: 2.8609e-04 - accuracy: 1.0000 - val_loss: 2.7719e-04 - val_accuracy: 1.0000
Epoch 4/100
1384001/1384001 [==============================] - 7s 5us/sample - loss: 2.7093e-04 - accuracy: 1.0000 - val_loss: 2.8273e-04 - val_accuracy: 1.0000
Epoch 5/100
1384001/1384001 [==============================] - 6s 5us/sample - loss: 2.9801e-04 - accuracy: 1.0000 - val_loss: 2.8455e-04 - val_accuracy: 1.0000
Epoch 6/100
1384001/1384001 [==============================] - 7s 5us/sample - loss: 3.0211e-04 - accuracy: 1.0000 - val_loss: 2.8973e-04 - val_ac

Epoch 47/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.6125e-04 - accuracy: 1.0000 - val_loss: 2.6722e-04 - val_accuracy: 1.0000
Epoch 48/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.6691e-04 - accuracy: 1.0000 - val_loss: 2.6742e-04 - val_accuracy: 1.0000
Epoch 49/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.8574e-04 - accuracy: 1.0000 - val_loss: 2.6661e-04 - val_accuracy: 1.0000
Epoch 50/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.7181e-04 - accuracy: 1.0000 - val_loss: 2.6662e-04 - val_accuracy: 1.0000
Epoch 51/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.6124e-04 - accuracy: 1.0000 - val_loss: 2.6712e-04 - val_accuracy: 1.0000
Epoch 52/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.7757e-04 - accuracy: 1.0000 - val_loss: 2.6678e-04 - val_accuracy: 1.0000
Epoch 53/100
1381376/1

Epoch 93/100
1382400/1384001 [============================>.] - ETA: 0s - loss: 2.6805e-04 - accuracy: 1.0000
Epoch 00093: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
1384001/1384001 [==============================] - 11s 8us/sample - loss: 2.6777e-04 - accuracy: 1.0000 - val_loss: 2.6654e-04 - val_accuracy: 1.0000
Epoch 94/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.6465e-04 - accuracy: 1.0000 - val_loss: 2.6654e-04 - val_accuracy: 1.0000
Epoch 95/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.6916e-04 - accuracy: 1.0000 - val_loss: 2.6655e-04 - val_accuracy: 1.0000
Epoch 96/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.7333e-04 - accuracy: 1.0000 - val_loss: 2.6654e-04 - val_accuracy: 1.0000
Epoch 97/100
1384001/1384001 [==============================] - 12s 8us/sample - loss: 2.7086e-04 - accuracy: 1.0000 - val_loss: 2.6654e-04 - val_accuracy: 1.0000
Epoch 9

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# Load the best saved model
best_model = load_model('best_model_binary.h5')

# Evaluate the best saved model
score = best_model.evaluate(x_val, y_val)
print('')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.summary()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 16))

ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='test')
ax.set_title('Model Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


def calculate_metrics(modelName, yTrue, yPred, average='binary'):
    """
    Calculate and print the performance metrics of a classification model.
    
    Parameters:
    modelName (str): The name of the classification model.
    yTrue (array-like): The true labels.
    yPred (array-like): The predicted labels.
    average (str or None, optional): The averaging method to use for multi-class classification. One of 
        {'micro', 'macro', 'weighted', 'binary'} or None (default: 'binary'). If None, only binary 
        classification metrics will be computed.
    
    Raises:
    ValueError: If `average` is not one of {'micro', 'macro', 'weighted', 'binary'} or None.
    
    """    
    # Check if average parameter is valid
    if average != 'micro' and average != 'macro' and average != 'weighted' and average != 'binary' and average != None:
        print("Average must be one of this options: {‘micro’, ‘macro’, ‘samples’, ‘weighted’, ‘binary’} or None, default=’binary’")
        return
    
    # Prints the name of the model and calculate accuracy and precision
    print(f"--- Performance of {modelName} ---")
    acc = accuracy_score(y_true = yTrue, y_pred = yPred)
    precision = precision_score(y_true = yTrue, y_pred = yPred, average = average)
    print(f'Accuracy : {np.round(acc*100,2)}%\nPrecision: {np.round(precision*100,2)}%')
    
    # Calculates and print recall and F1-score
    f1 = f1_score(y_true = yTrue, y_pred = yPred, average = average)
    recall = recall_score(y_true = yTrue, y_pred = yPred, average = average)
    print(f'Recall: {np.round(recall*100,2)}%\nF1-score: {np.round(f1*100,2)}%')
    
    #auc_sklearn = roc_auc_score(y_true = yTrue, y_score = yPred, average = average)
    #print(f'Roc auc: {np.round(auc_sklearn*100,2)}%')
    
    # Calculates and prints balanced accuracy and classification report
    print(f"Balanced accuracy: {np.round(balanced_accuracy_score(yTrue, yPred)*100,2)}%")
    print(f"Classification report:\n{classification_report(yTrue, yPred)}")

In [ ]:
pred = np.round(pred).astype(int)
calculate_metrics("Binary - DNN", y_test, pred)

-------------------------------------

**Result Plots**

In [ ]:
# Compute confusion matrix
y_pred = np.round(pred).astype(int)
cm = confusion_matrix(y_test, y_pred)

# Display confusion matrix
cmd = ConfusionMatrixDisplay(cm)
fig, ax = plt.subplots(figsize=(10, 10))
cmd.plot(ax=ax)

In [ ]:
# Usage of ExtraTreesClassifier for feature selection
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5, criterion ='entropy', max_features = 2)
extra_tree_forest.fit(x, y)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in  extra_tree_forest.estimators_], axis = 0)

In [ ]:
# Plor for the ExtraTreesClassifier output
plot.bar(x_columns, feature_importance_normalized)
plot.xlabel('Feature Labels')
plot.ylabel('Feature Importances')
plot.title('Comparison of different feature importances in the current dataset')
plot.xticks(rotation = 90)

# Plot size
plot.rcParams["figure.figsize"] = (70, 40)

plot.show()

## 